In [1]:
import pandas as pd
from scipy.stats import norm
from numpy import exp, log, sqrt

# === Black-Scholes helper functions ===

def calculate_d1(S, K, r, T, sigma):
    return (log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))

def delta_put(d1):
    return norm.cdf(d1) - 1

def delta_puts(n_puts, delta_1P):
    return n_puts * delta_1P

def delta_total(delta_puts, stock_position):
    return delta_puts + stock_position

def accrue_cash(cash_prev, r, dt_years):
    return cash_prev * exp(r * dt_years)

# === Portfolio parameters ===
K = 40
r = 0.02
sigma = 0.30
n_puts = 2000
T0 = 0.25
dt = 1 / 52

# Initial portfolio state (Week 0 BH)
df = pd.DataFrame(columns=[
    'options position', 'asset position', 'cash position', 'asset price', 'T',
    r'$\Delta_{1P}$', r'$\Delta_{puts}$', r'$\Delta_{total}$'
])

# Week 0 BH values
S = 35
stock = 400
cash = 10000
T = T0

d1 = calculate_d1(S, K, r, T, sigma)
delta1P = delta_put(d1)
delta_put_total = delta_puts(n_puts, delta1P)
delta_total_val = delta_total(delta_put_total, stock)

df.loc["week 0 BH"] = [n_puts, stock, cash, S, T, delta1P, delta_put_total, delta_total_val]

# Week 0 AH
stock += round(-delta_total_val)
cash -= (stock - df.loc["week 0 BH", "asset position"]) * S
d1 = calculate_d1(S, K, r, T, sigma)
delta1P = delta_put(d1)
delta_put_total = delta_puts(n_puts, delta1P)
delta_total_val = delta_total(delta_put_total, stock)
df.loc["week 0 AH"] = [n_puts, stock, cash, S, T, delta1P, delta_put_total, delta_total_val]

# Define price path
price_path = [40, 36, 32, 37]

# Simulate Weeks 1–4
for week, S in enumerate(price_path, start=1):
    prev_ah = f"week {week - 1} AH"
    bh = f"week {week} BH"
    ah = f"week {week} AH"
    
    T = df.loc[prev_ah, "T"] - dt
    stock = df.loc[prev_ah, "asset position"]
    cash = accrue_cash(df.loc[prev_ah, "cash position"], r, dt)
    
    d1 = calculate_d1(S, K, r, T, sigma)
    delta1P = delta_put(d1)
    delta_put_total = delta_puts(n_puts, delta1P)
    delta_total_val = delta_total(delta_put_total, stock)
    
    df.loc[bh] = [n_puts, stock, cash, S, T, delta1P, delta_put_total, delta_total_val]
    
    # Rebalance to neutralize delta
    stock += round(-delta_total_val)
    cash -= (stock - df.loc[bh, "asset position"]) * S
    
    d1 = calculate_d1(S, K, r, T, sigma)
    delta1P = delta_put(d1)
    delta_put_total = delta_puts(n_puts, delta1P)
    delta_total_val = delta_total(delta_put_total, stock)
    
    df.loc[ah] = [n_puts, stock, cash, S, T, delta1P, delta_put_total, delta_total_val]

In [2]:
print(df.to_latex(escape=False, index=True, float_format="%.4f"))

\begin{tabular}{lrrrrrrrr}
\toprule
 & options position & asset position & cash position & asset price & T & $\Delta_{1P}$ & $\Delta_{puts}$ & $\Delta_{total}$ \\
\midrule
week 0 BH & 2000.0000 & 400.0000 & 10000.0000 & 35.0000 & 0.2500 & -0.7829 & -1565.7125 & -1165.7125 \\
week 0 AH & 2000.0000 & 1566.0000 & -30810.0000 & 35.0000 & 0.2500 & -0.7829 & -1565.7125 & 0.2875 \\
week 1 BH & 2000.0000 & 1566.0000 & -30821.8523 & 40.0000 & 0.2308 & -0.4586 & -917.1032 & 648.8968 \\
week 1 AH & 2000.0000 & 917.0000 & -4861.8523 & 40.0000 & 0.2308 & -0.4586 & -917.1032 & -0.1032 \\
week 2 BH & 2000.0000 & 917.0000 & -4863.7226 & 36.0000 & 0.2115 & -0.7466 & -1493.2718 & -576.2718 \\
week 2 AH & 2000.0000 & 1493.0000 & -25599.7226 & 36.0000 & 0.2115 & -0.7466 & -1493.2718 & -0.2718 \\
week 3 BH & 2000.0000 & 1493.0000 & -25609.5705 & 32.0000 & 0.1923 & -0.9453 & -1890.6535 & -397.6535 \\
week 3 AH & 2000.0000 & 1891.0000 & -38345.5705 & 32.0000 & 0.1923 & -0.9453 & -1890.6535 & 0.3465 \\
week 4